In [ ]:
!pip install geemap --upgrade
!pip install earthengine-api --upgrade

In [ ]:
import ee

In [ ]:
import geemap

In [ ]:
Map = geemap.Map()
Map

In [ ]:
Map_1 = geemap.Map()
point = ee.Geometry.Point([29.3640, 41.0627])

image = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
    .filterBounds(point)  \
    .filterDate("2018-01-01", "2019-12-31") \
    .sort("CLOUD_COVER") \
    .first()  \
    .select("B[1-7]")

vis_parameters = {"min": 0, "max": 3000, "bands": ["B4", "B3", "B2"]}

Map_1.centerObject(point, 8)
Map_1.addLayer(image, vis_parameters, "Landsat-8")
Map_1

In [ ]:
ee.Algorithms.If(ee.List(image.propertyNames()).contains("system:time_start"), True, False).getInfo()

In [ ]:
props = geemap.image_props(image)
props.getInfo()

In [ ]:
props.get("IMAGE_DATE").getInfo()

In [ ]:
props.get("CLOUD_COVER").getInfo()

In [ ]:
#Map_1.user_roi.getInfo()

In [ ]:
#region = Map_1.user_roi

In [ ]:
training = image.sample(**{
#    "region": region,
    "scale": 50,
    "numPixels": 5000,
    "seed": 0,
    "geometries": True
})
Map_1.addLayer(training, {}, "training", False)
Map_1

In [ ]:
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [ ]:
result= image.cluster(clusterer)

Map_1.addLayer(result.randomVisualizer(), {}, "clusters")
Map_1

In [ ]:
legend_keys = ["Water", "Agricultural", "Rocky", "Settlement", "Forest"]
legend_colors = ["#8DD3C7","#FFFFB3", "#BEBADA", "#FB8072", "#80B1D3"]
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map_1.addLayer(result, {"min": 1, "max": 5, "palette": legend_colors}, "Labelled Clusters")
Map_1.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, poisiton="bottomright")
Map_1

In [ ]:
print("Change Layer Opacity:")
cluster_layer = Map_1.layers[-1]
cluster_layer.interact(opacity = (0, 1, 0.1))